# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

import gmaps.geojson_geometries
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacay = pd.read_csv("output_data/cities.csv", encoding="utf-8")
#vacay.reset_index()
vacay

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed
0,0,Aasiaat,100,GL,1647207422,93,68.7098,-52.8699,-0.42,7.90
1,1,Dikson,100,RU,1647207422,97,73.5069,80.5464,-5.26,16.64
2,2,Amoy,0,CN,1647207423,88,24.4798,118.0819,66.25,2.24
3,3,New Norfolk,100,AU,1647207423,67,-42.7826,147.0587,62.10,1.01
4,4,Alihe,100,CN,1647207423,97,50.5667,123.7167,16.03,1.48
...,...,...,...,...,...,...,...,...,...,...
547,547,Itoman,20,JP,1647207600,96,26.1247,127.6694,66.97,3.44
548,548,Kisanga,99,TZ,1647207601,88,-7.4362,37.7034,73.29,1.81
549,549,Uyuni,0,BO,1647207601,25,-20.4597,-66.8250,58.98,11.77
550,550,Dātāganj,0,IN,1647207602,39,28.0333,79.4000,67.37,2.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = vacay[["Lat", "Lng"]]
locations

,Lat,Lng
0,68.7098,-52.8699
1,73.5069,80.5464
2,24.4798,118.0819
3,-42.7826,147.0587
4,50.5667,123.7167
...,...,...
547,26.1247,127.6694
548,-7.4362,37.7034
549,-20.4597,-66.8250
550,28.0333,79.4000


In [5]:
humidity = vacay[["Humidity"]]
humidity= humidity['Humidity'].astype(float)
#humidity= pd.to_numeric(humidity['Humidity'], downcast='float')
#humidity = pd.DataFrame(humidity)
#humidity

In [6]:
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [7]:
fig = gmaps.figure(layout=figure_layout, zoom_level=2,center=(20,40))

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [8]:
fig = gmaps.figure(layout=figure_layout, zoom_level=2,center=(20,40))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#my ideal temperature for a vacation is pretty different than the example!

#A max temperature between 80 and 100 degrees
ideal_vacay = vacay[vacay["Temp"].between(80,100, inclusive=True)]

#Humidity greater than or equal to 50%
ideal_vacay = ideal_vacay[ideal_vacay["Humidity"] >=50]

#low cloudiness
ideal_vacay = ideal_vacay[ideal_vacay["Cloudiness"] <15]

#there was a city with a wind speed of 13... I'm not superstitious, but I am a little stitious
ideal_vacay = ideal_vacay[ideal_vacay["Wind Speed"] <13]

ideal_vacay.dropna(how='any')
ideal_vacay.reset_index(inplace=True)
del ideal_vacay['index']
ideal_vacay

/var/folders/gr/g0lzn1p15gqfb11105yyt_tm0000gn/T/ipykernel_73998/3008660479.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  ideal_vacay = vacay[vacay["Temp"].between(80,100, inclusive=True)]


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed
0,25,Hilo,0,US,1647207431,62,19.7297,-155.0900,82.13,11.50
1,127,Kieta,13,PG,1647207463,68,-6.2167,155.6333,82.08,2.35
2,180,Alofi,11,NU,1647207480,74,-19.0595,-169.9187,84.09,6.91
3,194,Karratha,3,AU,1647207485,68,-20.7377,116.8463,84.92,8.55
4,294,Roebourne,2,AU,1647207517,64,-20.7833,117.1333,84.96,8.03
5,299,Morant Bay,4,JM,1647207519,68,17.8815,-76.4093,88.72,12.68
6,426,Lucea,9,JM,1647207561,72,18.4510,-78.1736,82.85,12.35
7,438,Barra,10,BR,1647207565,51,-11.0894,-43.1417,82.35,0.94
8,539,Lázaro Cárdenas,0,MX,1647207426,55,17.9583,-102.2000,81.95,9.73


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = ideal_vacay
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed
0,25,Hilo,0,US,1647207431,62,19.7297,-155.0900,82.13,11.50
1,127,Kieta,13,PG,1647207463,68,-6.2167,155.6333,82.08,2.35
2,180,Alofi,11,NU,1647207480,74,-19.0595,-169.9187,84.09,6.91
3,194,Karratha,3,AU,1647207485,68,-20.7377,116.8463,84.92,8.55
4,294,Roebourne,2,AU,1647207517,64,-20.7833,117.1333,84.96,8.03
5,299,Morant Bay,4,JM,1647207519,68,17.8815,-76.4093,88.72,12.68
6,426,Lucea,9,JM,1647207561,72,18.4510,-78.1736,82.85,12.35
7,438,Barra,10,BR,1647207565,51,-11.0894,-43.1417,82.35,0.94
8,539,Lázaro Cárdenas,0,MX,1647207426,55,17.9583,-102.2000,81.95,9.73


In [11]:
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed,Hotel Name
0,25,Hilo,0,US,1647207431,62,19.7297,-155.0900,82.13,11.50,
1,127,Kieta,13,PG,1647207463,68,-6.2167,155.6333,82.08,2.35,
2,180,Alofi,11,NU,1647207480,74,-19.0595,-169.9187,84.09,6.91,
3,194,Karratha,3,AU,1647207485,68,-20.7377,116.8463,84.92,8.55,
4,294,Roebourne,2,AU,1647207517,64,-20.7833,117.1333,84.96,8.03,
5,299,Morant Bay,4,JM,1647207519,68,17.8815,-76.4093,88.72,12.68,
6,426,Lucea,9,JM,1647207561,72,18.4510,-78.1736,82.85,12.35,
7,438,Barra,10,BR,1647207565,51,-11.0894,-43.1417,82.35,0.94,
8,539,Lázaro Cárdenas,0,MX,1647207426,55,17.9583,-102.2000,81.95,9.73,


In [12]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Lat']
    lng = row['Lng']
    
    try:
        params = {
            'location': f"{lat},{lng}",
            'keyword': 'hotel',
            'radius': 5000,
            'key': g_key
        }
        
        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df       

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind Speed,Hotel Name
0,25,Hilo,0,US,1647207431,62,19.7297,-155.0900,82.13,11.50,Hilo Hawaiian Hotel
1,127,Kieta,13,PG,1647207463,68,-6.2167,155.6333,82.08,2.35,NaN
2,180,Alofi,11,NU,1647207480,74,-19.0595,-169.9187,84.09,6.91,Kololi's Motel
3,194,Karratha,3,AU,1647207485,68,-20.7377,116.8463,84.92,8.55,Karratha International Hotel
4,294,Roebourne,2,AU,1647207517,64,-20.7833,117.1333,84.96,8.03,Latitude20 Roebourne Village
5,299,Morant Bay,4,JM,1647207519,68,17.8815,-76.4093,88.72,12.68,Golden Shore Resort
6,426,Lucea,9,JM,1647207561,72,18.4510,-78.1736,82.85,12.35,Grand Palladium Lady Hamilton Resort & Spa
7,438,Barra,10,BR,1647207565,51,-11.0894,-43.1417,82.35,0.94,Hotel Canoas
8,539,Lázaro Cárdenas,0,MX,1647207426,55,17.9583,-102.2000,81.95,9.73,Hotel de Casablanca


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…